# Welcome to Aurite Agents!

This tutorial is the first in a series designed to guide you through building and running your own AI agents. By the end of this notebook, you will have configured and run a simple AI agent that can answer a question using a powerful Large Language Model (LLM).

### So, what is Aurite Agents?

Think of it as a toolkit for building with AI. It's a Python framework that provides the building blocks to create, manage, and deploy AI agents. It handles the complex parts of interacting with LLMs and external tools, so you can focus on the creative part: defining what your agent does.

## 1. Setup

### Install the `aurite` Python package

First, we'll install the `aurite` package. For this tutorial, we assume you have built the package from source. We will install the wheel file from the `dist` directory.

In [39]:
%pip install ../../../dist/aurite-0.3.26-py3-none-any.whl --force-reinstall

Processing /home/wilcoxr/workspace/aurite/framework/dist/aurite-0.3.26-py3-none-any.whl
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached cryptography-45.0.5-cp311-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
  Using cached exceptiongroup-1.3.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached fastapi-0.115.14-py3-none-any.whl.metadata (27 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached litellm-1.74.1-py3-none-any.whl.metadata (40 kB)
  Using cached mcp-1.11.0-py3-none-any.whl.metadata (44 kB)
  Using cached openai-1.95.0-py3-none-any.whl.metadata (29 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached pytz-2025.2-p

### Configure Your API Key

To use a Large Language Model like OpenAI's GPT-4, you need an API key. For security, it's best practice to set this as an environment variable rather than writing it directly in your code.

**Obtain an OpenAI API Key:**

*   This tutorial uses an OpenAI model (GPT-4 Turbo). To interact with it, you'll need an API key from OpenAI.
*   Navigate to https://platform.openai.com/api-keys to create or retrieve your API key.

> ⚠️ **Important: Protect Your Keys**
> Your API key is a secret. Treat it like a password. Exposing it can lead to unauthorized use and unexpected charges on your account.

**How to set it up:**
*   **Google Colab Users:** Use the **Secrets** tab (the key icon 🔑 on the left sidebar). Create a new secret named `OPENAI_API_KEY`, paste your key as the value, and make sure to enable it for this notebook.
*   **Local IDE Users (like VS Code):** Create a `.env` file in your root workspace folder (the one you opened in your IDE) and add the line `OPENAI_API_KEY=sk-your-key-here`.

In [40]:
import os
from getpass import getpass

os.environ["AURITE_CONFIG_FORCE_REFRESH"] = "false"  # Disable config force refresh to allow in-memory component registration

try:
  from google.colab import userdata #type: ignore
  os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
except Exception as e:
    print(f"Error fetching Secrets: {e}")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

## 🚀 2. Initialize Aurite & Create Your First Agent

First, we need to initialize the `Aurite` class. This is the main entry point for your project. When working in a notebook outside of a formal project structure, it's best to programmatically define and register our components.

In [41]:
from aurite import Aurite, AgentConfig, LLMConfig

# In a notebook, it's often easier to define configurations in code.
# For real projects, you would save these as .json or .yaml files.

# First, let's define a default LLM config for our agents to use.
default_llm = LLMConfig(
    name="default_llm",
    provider="anthropic",
    model="claude-3-sonnet-20240229",
)

# Initialize Aurite. When working outside of a project context, it's good practice
# to disable file-based logging to keep the output clean.
aurite = Aurite(disable_logging=True)

# Register our default LLM so other components can reference it.
await aurite.register_llm(default_llm)

print("✅ Aurite initialized and default LLM registered!")

✅ Aurite initialized and default LLM registered!


### Building a Basic Agent

Let's build our first **`Agent`**. We'll give it a `name` and tell it which LLM configuration to use with `llm_config_id`. Without any special instructions or tools, it behaves just like a standard chatbot.

In [42]:
# Create an agent configuration, linking it to our default LLM.
agent_config = AgentConfig(
    name="My First Agent",
    llm_config_id="default_llm"
)

# Register the agent with Aurite so the framework knows about it.
await aurite.register_agent(agent_config)

print("✅ Agent registered successfully! You can now interact with it.")

✅ Agent registered successfully! You can now interact with it.


### Run the Agent!

Now, let's ask our new agent a question.

In [43]:
user_message = "Hello! Can you tell me a joke?"

# Run the agent with our query
agent_result = await aurite.run_agent(
    agent_name="My First Agent",
    user_message=user_message
)

print("Agent Response:")
print(agent_result.primary_text)

Agent Response:
Sure, here's a joke for you:

Why can't a bicycle stand up by itself? It's two-tired!


### A Helper for Better Output

Let's create a helper function to make the output look much nicer.

In [44]:
from IPython.display import display, Markdown

def display_agent_response(agent_name: str, query: str, response: str = "No response provided"):
  """Formats and displays the agent's response in a structured Markdown block."""

  output = f"""
  <div style=\"border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);\">
    <div style=\"background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;\">
      <h3 style=\"margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;\">
        <span style=\"margin-right: 8px;\">🤖</span>
        Agent Response: <code style=\"background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;\">{agent_name}</code>
      </h3>
    </div>
    <div style=\"padding: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Your Query:</strong>
      </p>
      <p style=\"background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;\">
        <em>\"{query}\"</em>
      </p>
      <hr style=\"border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Result:</strong>
      </p>
      <div style=\"background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;\">
        {response}
      </div>
    </div>
  </div>
  """
  display(Markdown(output))

In [45]:
# Now let's display the result from our first agent run using the new helper!
display_agent_response(
    agent_name="My First Agent",
    query=user_message,
    response=agent_result.primary_text if agent_result.primary_text else "No response received."
)


  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">My First Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"Hello! Can you tell me a joke?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        Sure, here's a joke for you:

Why can't a bicycle stand up by itself? It's two-tired!
      </div>
    </div>
  </div>
  

## 🎭 3. Giving Your Agent a Personality

The previous example was just a plain chatbot. The real power comes from giving your agent a specific role and task. The most important tool for this is the **`system_prompt`**.

### What is a System Prompt?

A **`system_prompt`** is a set of instructions you give the LLM that defines its personality, its goal, and the rules it must follow. Think of yourself as a director and the agent as an improv actor. The system prompt is your direction.

> 💡 **Pro-Tip: A Good System Prompt Template**
> A great starting point for any system prompt follows this structure:
> - **Role:** "You are a..."
> - **Task:** "Your job is to..."
> - **Context:** "In order to do this..."
> - **Rules:** "You MUST... / You MUST NOT..."

In [46]:
# Let's create a new agent with a dramatic personality.
system_prompt = "You are a helpful but overly dramatic assistant. " \
    "Your job is to answer the user's question in an overly dramatic tone. " \
    "You must only respond with the answer to the user's question in a dramatic way. Add flair before and after your response, like a Shakespearean actor. "

# Create a new agent configuration with the system prompt
dramatic_agent = AgentConfig(
    name="My Dramatic Agent",
    system_prompt=system_prompt,
    llm_config_id="default_llm" # Don't forget to link the LLM!
)

# Register the new agent
await aurite.register_agent(dramatic_agent)

# Run the agent with the same user message as before
agent_result = await aurite.run_agent(
    agent_name="My Dramatic Agent",
    user_message=user_message # Notice how the response will be different now!
)

# Display the dramatic response
display_agent_response(
    agent_name="My Dramatic Agent",
    query=user_message,
    response=agent_result.primary_text if agent_result.primary_text else "No response received."
)


  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">My Dramatic Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"Hello! Can you tell me a joke?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        *clears throat dramatically* Ah, a joke you request? Let me regale thee with a tale most humorous!

*throws cape over shoulder* Why did the scarecrow win an award? Because he was outstanding in his field! *bellows with laughter, then composes self* 
      </div>
    </div>
  </div>
  

## 🛠️ 4. Giving Your Agent Tools

An agent with a personality is fun, but an agent that can *do* things is powerful. This is where **`Tools`** come in.

### What are Tools?

**Tools are functions (executable code) that an agent can decide to use to get information or perform an action.** This is what gives an agent true *agency*—the ability to act. Without tools, an LLM can only talk. With tools, it can interact with the world.

The Aurite framework uses the **Model Context Protocol (MCP)** to define and manage tools. You can write your own tool servers, but the framework also comes with several pre-built ones.

> 🔗 **Further Reading**
> Tools are a deep topic! To learn more about how they work, check out the documentation on **[MCP Servers](https://aurite.ai/docs/config/mcp_server)**.

Let's give an agent a (fake) weather tool. We first need to register the tool server, then tell the agent it has access to it via the **`mcp_servers`** property.

In [ ]:
from aurite import ClientConfig

# This config points to a pre-packaged python script that acts as a tool server.
weather_server_config = ClientConfig(
    name="weather_server",
    server_path="../../../src/aurite/lib/init_templates/mcp_servers/weather_server.py",  # This path is relative to the project root
    capabilities=["tools"],  # Ensure the server supports tools
)

await aurite.register_mcp_server(weather_server_config)

system_prompt = "You are a Weather Forecaster. " \
    "Your job is to provide accurate weather information based on the user's query. " \
    "In order to do this, you must use the weather tool at your disposal. " \
    "You must only respond with the weather forecast for the location specified by the user."

user_message = "What's the weather like in New York?"

# Define and register an Agent configuration with tools
agent_config = AgentConfig(
    name="My Weather Agent",
    system_prompt=system_prompt,
    llm_config_id="default_llm",
    mcp_servers=["weather_server"]
)

await aurite.register_agent(agent_config)

# Run the weather agent
agent_result = await aurite.run_agent(
    agent_name="My Weather Agent",
    user_message=user_message
)

# Display the final forecast
display_agent_response(
    agent_name="My Weather Agent",
    query=user_message,
    response=agent_result.primary_text if agent_result.primary_text else "No response received."
)


  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">My Weather Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"What's the weather like in New York?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        The current weather in New York is 72°F, partly cloudy, with 60% humidity.
      </div>
    </div>
  </div>
  

> ⚠️ **Note on the Example Tool**
> The `weather_server` used here is a simplified, built-in example for demonstration purposes. It will always return a pre-defined forecast, and it only supports London, San Francisco, New York, and Tokyo. In the next tutorial, we'll use real tools!

## 🎉 Congratulations!

You've successfully built and run your first few AI agents using the Aurite framework.

**Recap:**

*   You learned how to configure an **`Agent`** with a **`system_prompt`** and a set of **`Tools`** (`mcp_servers`).
*   You initialized the `Aurite` application and registered your components.
*   You ran agents with different personalities and capabilities.

## Next Steps

In the next tutorial, we will explore how to give agents real tools to interact with the world using MCP Servers.

**[➡️ Open Tutorial 2: Agents and Tools](https://colab.research.google.com/drive/1Umm0TUSlgInpZQqVjWiBpvn5AUJGDkAo?usp=sharing)**